In [1]:
import netCDF4 as nc4
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, add
from keras.layers.core import  Dense, Dropout, Activation, Flatten, Reshape
from keras import regularizers
from keras.regularizers import l2
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.utils import np_utils

In [2]:
dataset = []
bytes_before_compression= []
bytes_after_compression = [] 
compression_ratio = []

In [3]:
file_path =['C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_01.nc']
# file_path.append('C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_02.nc')
# file_path.append('C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_03.nc')
# file_path.append('C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_04.nc')
# file_path.append('C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_05.nc')
# file_path.append('C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_06.nc')

In [4]:
ds = nc4.MFDataset(file_path)
eta_t_arr = ds.variables['eta_t'][:]
time_ytocean_arr = eta_t_arr[:,:,0]
time_ytocean_arr.shape
time_ytocean_arr = np.divide(time_ytocean_arr, 2)
time_ytocean_arr[0]
for i in range(0,len(time_ytocean_arr)):
    arr = time_ytocean_arr[i].data
    arr[arr == -16384] = 0
time_ytocean_train, time_ytocean_test = train_test_split(time_ytocean_arr.data, test_size=0.33)
time_ytocean_train.shape
input_size = 1500
hidden_size = 150
output_size = 1500
x = Input(shape=(input_size,))
h = Dense(hidden_size, activation='tanh')(x)
r = Dense(output_size, activation='tanh')(h)

autoencoder = Model(inputs=x, outputs=r)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()
epochs = 30
batch_size = 1500

history = autoencoder.fit(time_ytocean_train, time_ytocean_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(time_ytocean_test, time_ytocean_test))

import sys
# encoder
encoder = Model(x,h)
predict  = encoder.predict(time_ytocean_test)
print(predict.shape)
print(time_ytocean_test.shape)
print(sys.getsizeof(time_ytocean_test))
sys.getsizeof(predict)
dataset.append('ocean_eta_t_2000_01.nc')
bytes_before_compression.append(sys.getsizeof(time_ytocean_test))
bytes_after_compression.append(sys.getsizeof(predict))
compression_ratio.append(round((sys.getsizeof(time_ytocean_test)/sys.getsizeof(predict)),4))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1500)]            0         
_________________________________________________________________
dense (Dense)                (None, 150)               225150    
_________________________________________________________________
dense_1 (Dense)              (None, 1500)              226500    
Total params: 451,650
Trainable params: 451,650
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.1593 - val_loss: 0.1229
Epoch 2/100
1/1 [==============================] - 0s 64ms/step - loss: 0.1238 - val_loss: 0.1087
Epoch 3/100
1/1 [==============================] - 0s 64ms/step - loss: 0.1096 - val_loss: 0.0980
Epoch 4/100
1/1 [==============================] - 0s 63ms/step - loss: 0.0988 - val_loss: 0.

In [19]:
compression_ratio

[9.8351]

In [11]:
import pandas as pd
df = pd.DataFrame({"dataset":dataset, "bytes_before_compression":bytes_before_compression,'bytes_after_compression':bytes_after_compression,'compression_ratio':compression_ratio}).reset_index(drop=True)
df

,dataset,bytes_before_compression,bytes_after_compression,compression_ratio
0,ocean_eta_t_2000_01.nc,66112,6712,9.8498
1,ocean_eta_t_2000_02.nc,60112,6112,9.8351
2,ocean_eta_t_2000_03.nc,66112,6712,9.8498
3,ocean_eta_t_2000_04.nc,60112,6112,9.8351
4,ocean_eta_t_2000_05.nc,66112,6712,9.8498
5,ocean_eta_t_2000_06.nc,60112,6112,9.8351


## er

In [13]:
time_ytocean_arr.shape

(29, 1500)

In [12]:
file_path =['C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_02.nc']
ds = nc4.MFDataset(file_path)
eta_t_arr = ds.variables['eta_t'][:]
time_ytocean_arr = eta_t_arr[:,:,0]
time_ytocean_arr.shape
time_ytocean_arr = np.divide(time_ytocean_arr, 2)
time_ytocean_arr[0]
for i in range(0,len(time_ytocean_arr)):
    arr = time_ytocean_arr[i].data
    arr[arr == -16384] = 0
time_ytocean_train, time_ytocean_test = train_test_split(time_ytocean_arr.data, test_size=0.33)
time_ytocean_train.shape
input_size = 1500
hidden_size = 150
output_size = 1500

x = Input(shape=(input_size,))
h = Dense(hidden_size, activation='tanh')(x)
r = Dense(output_size, activation='tanh')(h)

autoencoder = Model(inputs=x, outputs=r)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()
epochs = 30
batch_size = 1500

history = autoencoder.fit(time_ytocean_train, time_ytocean_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(time_ytocean_test, time_ytocean_test))

encoder = Model(x,h)
predict  = encoder.predict(time_ytocean_test)
print(predict.shape)
print(time_ytocean_test.shape)
print(sys.getsizeof(time_ytocean_test))
sys.getsizeof(predict)
dataset.append('ocean_eta_t_2000_02.nc')
bytes_before_compression.append(sys.getsizeof(time_ytocean_test))
bytes_after_compression.append(sys.getsizeof(predict))
compression_ratio.append(round((sys.getsizeof(time_ytocean_test)/sys.getsizeof(predict)),4))

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 1500)]            0         
_________________________________________________________________
dense_12 (Dense)             (None, 150)               225150    
_________________________________________________________________
dense_13 (Dense)             (None, 1500)              226500    
Total params: 451,650
Trainable params: 451,650
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 1s 706ms/step - loss: 0.1509 - val_loss: 0.1205
Epoch 2/100
1/1 [==============================] - 0s 63ms/step - loss: 0.1208 - val_loss: 0.1049
Epoch 3/100
1/1 [==============================] - 0s 65ms/step - loss: 0.1052 - val_loss: 0.0887
Epoch 4/100
1/1 [==============================] - 0s 63ms/step - loss: 0.0888 - val_lo

## san


In [7]:
file_path =['C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_03.nc']

ds = nc4.MFDataset(file_path)
eta_t_arr = ds.variables['eta_t'][:]
time_ytocean_arr = eta_t_arr[:,:,0]
time_ytocean_arr.shape
time_ytocean_arr = np.divide(time_ytocean_arr, 2)
time_ytocean_arr[0]
for i in range(0,len(time_ytocean_arr)):
    arr = time_ytocean_arr[i].data
    arr[arr == -16384] = 0
time_ytocean_train, time_ytocean_test = train_test_split(time_ytocean_arr.data, test_size=0.33)
time_ytocean_train.shape
input_size = 1500
hidden_size = 150
output_size = 1500

x = Input(shape=(input_size,))
h = Dense(hidden_size, activation='tanh')(x)
r = Dense(output_size, activation='tanh')(h)

autoencoder = Model(inputs=x, outputs=r)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()
epochs = 100
batch_size = 1500

history = autoencoder.fit(time_ytocean_train, time_ytocean_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(time_ytocean_test, time_ytocean_test))

encoder = Model(x,h)
predict  = encoder.predict(time_ytocean_test)
print(predict.shape)
print(time_ytocean_test.shape)
print(sys.getsizeof(time_ytocean_test))
sys.getsizeof(predict)
dataset.append('ocean_eta_t_2000_03.nc')
bytes_before_compression.append(sys.getsizeof(time_ytocean_test))
bytes_after_compression.append(sys.getsizeof(predict))
compression_ratio.append(round((sys.getsizeof(time_ytocean_test)/sys.getsizeof(predict)),4))

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1500)]            0         
_________________________________________________________________
dense_4 (Dense)              (None, 150)               225150    
_________________________________________________________________
dense_5 (Dense)              (None, 1500)              226500    
Total params: 451,650
Trainable params: 451,650
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 1s 671ms/step - loss: 0.1650 - val_loss: 0.1318
Epoch 2/100
1/1 [==============================] - 0s 69ms/step - loss: 0.1317 - val_loss: 0.1165
Epoch 3/100
1/1 [==============================] - 0s 68ms/step - loss: 0.1165 - val_loss: 0.1032
Epoch 4/100
1/1 [==============================] - 0s 71ms/step - loss: 0.1032 - val_los

## si

In [8]:
file_path =['C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_04.nc']
ds = nc4.MFDataset(file_path)
eta_t_arr = ds.variables['eta_t'][:]
time_ytocean_arr = eta_t_arr[:,:,0]
time_ytocean_arr.shape
time_ytocean_arr = np.divide(time_ytocean_arr, 2)
time_ytocean_arr[0]
for i in range(0,len(time_ytocean_arr)):
    arr = time_ytocean_arr[i].data
    arr[arr == -16384] = 0
time_ytocean_train, time_ytocean_test = train_test_split(time_ytocean_arr.data, test_size=0.33)
time_ytocean_train.shape
input_size = 1500
hidden_size = 150
output_size = 1500

x = Input(shape=(input_size,))
h = Dense(hidden_size, activation='tanh')(x)
r = Dense(output_size, activation='tanh')(h)

autoencoder = Model(inputs=x, outputs=r)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()
epochs = 100
batch_size = 1500

history = autoencoder.fit(time_ytocean_train, time_ytocean_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(time_ytocean_test, time_ytocean_test))

encoder = Model(x,h)
predict  = encoder.predict(time_ytocean_test)
print(predict.shape)
print(time_ytocean_test.shape)
print(sys.getsizeof(time_ytocean_test))
sys.getsizeof(predict)
dataset.append('ocean_eta_t_2000_04.nc')
bytes_before_compression.append(sys.getsizeof(time_ytocean_test))
bytes_after_compression.append(sys.getsizeof(predict))
compression_ratio.append(round((sys.getsizeof(time_ytocean_test)/sys.getsizeof(predict)),4))

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1500)]            0         
_________________________________________________________________
dense_6 (Dense)              (None, 150)               225150    
_________________________________________________________________
dense_7 (Dense)              (None, 1500)              226500    
Total params: 451,650
Trainable params: 451,650
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 1s 585ms/step - loss: 0.1681 - val_loss: 0.1300
Epoch 2/100
1/1 [==============================] - 0s 69ms/step - loss: 0.1302 - val_loss: 0.1105
Epoch 3/100
1/1 [==============================] - 0s 76ms/step - loss: 0.1106 - val_loss: 0.0952
Epoch 4/100
1/1 [==============================] - 0s 66ms/step - loss: 0.0953 - val_los

In [9]:
file_path =['C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_05.nc']
ds = nc4.MFDataset(file_path)
eta_t_arr = ds.variables['eta_t'][:]
time_ytocean_arr = eta_t_arr[:,:,0]
time_ytocean_arr.shape
time_ytocean_arr = np.divide(time_ytocean_arr, 2)
time_ytocean_arr[0]
for i in range(0,len(time_ytocean_arr)):
    arr = time_ytocean_arr[i].data
    arr[arr == -16384] = 0
time_ytocean_train, time_ytocean_test = train_test_split(time_ytocean_arr.data, test_size=0.33)
time_ytocean_train.shape
input_size = 1500
hidden_size = 150
output_size = 1500

x = Input(shape=(input_size,))
h = Dense(hidden_size, activation='tanh')(x)
r = Dense(output_size, activation='tanh')(h)

autoencoder = Model(inputs=x, outputs=r)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()
epochs = 100
batch_size = 1500

history = autoencoder.fit(time_ytocean_train, time_ytocean_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(time_ytocean_test, time_ytocean_test))

encoder = Model(x,h)
predict  = encoder.predict(time_ytocean_test)
print(predict.shape)
print(time_ytocean_test.shape)
print(sys.getsizeof(time_ytocean_test))
sys.getsizeof(predict)
dataset.append('ocean_eta_t_2000_05.nc')
bytes_before_compression.append(sys.getsizeof(time_ytocean_test))
bytes_after_compression.append(sys.getsizeof(predict))
compression_ratio.append(round((sys.getsizeof(time_ytocean_test)/sys.getsizeof(predict)),4))

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1500)]            0         
_________________________________________________________________
dense_8 (Dense)              (None, 150)               225150    
_________________________________________________________________
dense_9 (Dense)              (None, 1500)              226500    
Total params: 451,650
Trainable params: 451,650
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 1s 564ms/step - loss: 0.1656 - val_loss: 0.1317
Epoch 2/100
1/1 [==============================] - 0s 64ms/step - loss: 0.1309 - val_loss: 0.1136
Epoch 3/100
1/1 [==============================] - 0s 62ms/step - loss: 0.1129 - val_loss: 0.0997
Epoch 4/100
1/1 [==============================] - 0s 76ms/step - loss: 0.0992 - val_los

In [10]:
file_path =['C:/Users/Administrator/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_06.nc']
ds = nc4.MFDataset(file_path)
eta_t_arr = ds.variables['eta_t'][:]
time_ytocean_arr = eta_t_arr[:,:,0]
time_ytocean_arr.shape
time_ytocean_arr = np.divide(time_ytocean_arr, 2)
time_ytocean_arr[0]
for i in range(0,len(time_ytocean_arr)):
    arr = time_ytocean_arr[i].data
    arr[arr == -16384] = 0
time_ytocean_train, time_ytocean_test = train_test_split(time_ytocean_arr.data, test_size=0.33)
time_ytocean_train.shape
input_size = 1500
hidden_size = 150
output_size = 1500

x = Input(shape=(input_size,))
h = Dense(hidden_size, activation='tanh')(x)
r = Dense(output_size, activation='tanh')(h)

autoencoder = Model(inputs=x, outputs=r)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()
epochs = 100
batch_size = 1500

history = autoencoder.fit(time_ytocean_train, time_ytocean_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(time_ytocean_test, time_ytocean_test))

encoder = Model(x,h)
predict  = encoder.predict(time_ytocean_test)
print(predict.shape)
print(time_ytocean_test.shape)
print(sys.getsizeof(time_ytocean_test))
sys.getsizeof(predict)
dataset.append('ocean_eta_t_2000_06.nc')
bytes_before_compression.append(sys.getsizeof(time_ytocean_test))
bytes_after_compression.append(sys.getsizeof(predict))
compression_ratio.append(round((sys.getsizeof(time_ytocean_test)/sys.getsizeof(predict)),4))

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1500)]            0         
_________________________________________________________________
dense_10 (Dense)             (None, 150)               225150    
_________________________________________________________________
dense_11 (Dense)             (None, 1500)              226500    
Total params: 451,650
Trainable params: 451,650
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 1s 530ms/step - loss: 0.1580 - val_loss: 0.1286
Epoch 2/100
1/1 [==============================] - 0s 68ms/step - loss: 0.1283 - val_loss: 0.1141
Epoch 3/100
1/1 [==============================] - 0s 62ms/step - loss: 0.1138 - val_loss: 0.1005
Epoch 4/100
1/1 [==============================] - 0s 63ms/step - loss: 0.1002 - val_lo